In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters

{'watershed': 'han_auto'}

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)
#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2015.xlsx', '가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx', '가평_2019.xlsx', '가평_2020.xlsx'], ['서상_2015.xlsx', '서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx', '서상_2019.xlsx', '서상_2020.xlsx'], ['의암호_2015.xlsx', '의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx', '의암호_2019.xlsx', '의암호_2020.xlsx']]
data/han/자동/가평_2015.xlsx
data/han/자동/가평_2016.xlsx
data/han/자동/가평_2017.xlsx
data/han/자동/가평_2018.xlsx
data/han/자동/가평_2019.xlsx
data/han/자동/가평_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/han/자동/서상_2015.xlsx
data/han/자동/서상_2016.xlsx
data/han/자동/서상_2017.xlsx
data/han/자동/서상_2018.xlsx
data/han/자동/서상_2019.xlsx
data/han/자동/서상_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/han/자동/의암호_2015.xlsx
data/han/자동/의암호_2016.xlsx
data/han/자

1/1 [==============================] - 0s 20ms/step - gen_loss: 14.9088 - disc_loss: 0.3602 - rmse: 0.4795 - val_loss: 0.4017
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.3307 - disc_loss: 0.3677 - rmse: 0.3970 - val_loss: 0.4230
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.5650 - disc_loss: 0.3523 - rmse: 0.4090 - val_loss: 0.4091
Epoch 53/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 12.5890 - disc_loss: 0.3698 - rmse: 0.4430 - val_loss: 0.4058
Epoch 54/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 16.2932 - disc_loss: 0.3595 - rmse: 0.4703 - val_loss: 0.3941
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.2170 - disc_loss: 0.3515 - rmse: 0.4218 - val_loss: 0.4681
Epoch 56/2000
1/1 [==============================] - 0s 28ms/step - gen_loss: 13.1154 - disc_loss: 0.3657 - rmse: 0.4464 - val_loss: 0.3943
Epoch 57/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 9.1235 - disc_loss: 0.3278 - rmse: 0.3903 - val_loss: 0.3832
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.8030 - disc_loss: 0.2875 - rmse: 0.3973 - val_loss: 0.3513
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.5429 - disc_loss: 0.2934 - rmse: 0.3526 - val_loss: 0.3834
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.9793 - disc_loss: 0.3127 - rmse: 0.3576 - val_loss: 0.3511
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.7218 - disc_loss: 0.2999 - rmse: 0.3879 - val_loss: 0.4102
Epoch 113/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 11.6793 - disc_loss: 0.2966 - rmse: 0.7468 - val_loss: 0.3915
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.9896 - disc_loss: 0.2937 - rmse: 0.3535 - val_loss: 0.3702
Epoch 115/2000
1/1 [============

1/1 [==============================] - 0s 20ms/step - gen_loss: 9.2049 - disc_loss: 0.2662 - rmse: 0.4108 - val_loss: 0.3316
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0184 - disc_loss: 0.2831 - rmse: 0.3716 - val_loss: 0.3196
Epoch 168/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5610 - disc_loss: 0.2763 - rmse: 0.3149 - val_loss: 0.3716
Epoch 169/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3016 - disc_loss: 0.2750 - rmse: 0.4224 - val_loss: 0.2919
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.8268 - disc_loss: 0.2836 - rmse: 0.3698 - val_loss: 0.3227
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2462 - disc_loss: 0.2617 - rmse: 0.3249 - val_loss: 0.3436
Epoch 172/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.0174 - disc_loss: 0.2769 - rmse: 0.3537 - val_loss: 0.3817
Epoch 173/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 7.1741 - disc_loss: 0.2628 - rmse: 0.3455 - val_loss: 0.2959
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.8563 - disc_loss: 0.2763 - rmse: 0.3482 - val_loss: 0.3287
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.4015 - disc_loss: 0.2570 - rmse: 0.3547 - val_loss: 0.3322
Epoch 227/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.1391 - disc_loss: 0.2508 - rmse: 0.2923 - val_loss: 0.3135
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.4092 - disc_loss: 0.2469 - rmse: 0.7758 - val_loss: 0.2961
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.5366 - disc_loss: 0.2557 - rmse: 0.3935 - val_loss: 0.3130
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8874 - disc_loss: 0.2607 - rmse: 0.3543 - val_loss: 0.3402
Epoch 231/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5599 - disc_loss: 0.1598 - rmse: 0.2917 - val_loss: 0.2642
Epoch 1211/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7825 - disc_loss: 0.1587 - rmse: 0.2592 - val_loss: 0.2872
Epoch 1212/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7833 - disc_loss: 0.1718 - rmse: 0.2835 - val_loss: 0.3098
Epoch 1213/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.3980 - disc_loss: 0.1666 - rmse: 0.2597 - val_loss: 0.3026
Epoch 1214/2000
1/1 [==============================] - 0s 19ms/step - gen_loss: 5.4921 - disc_loss: 0.1569 - rmse: 0.2928 - val_loss: 0.7964
Epoch 1215/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1665 - disc_loss: 0.1717 - rmse: 0.2686 - val_loss: 0.3178
Epoch 1216/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3672 - disc_loss: 0.1507 - rmse: 0.2767 - val_loss: 0.2845
Epoch 1217/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0566 - disc_loss: 0.1576 - rmse: 0.2844 - val_loss: 0.2895
Epoch 1269/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 4.7990 - disc_loss: 0.1640 - rmse: 0.2443 - val_loss: 0.3109
Epoch 1270/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8739 - disc_loss: 0.1651 - rmse: 0.2392 - val_loss: 0.3218
Epoch 1271/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.9877 - disc_loss: 0.1653 - rmse: 0.2665 - val_loss: 0.3084
Epoch 1272/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2342 - disc_loss: 0.1616 - rmse: 0.2257 - val_loss: 0.2936
Epoch 1273/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.9394 - disc_loss: 0.1639 - rmse: 0.2978 - val_loss: 0.2617
Epoch 1274/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9988 - disc_loss: 0.1720 - rmse: 0.2720 - val_loss: 0.3699
Epoch 1275/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3186 - disc_loss: 0.1590 - rmse: 0.2692 - val_loss: 0.3087
Epoch 1327/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3160 - disc_loss: 0.1656 - rmse: 0.2605 - val_loss: 0.2802
Epoch 1328/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7104 - disc_loss: 0.1687 - rmse: 0.2385 - val_loss: 0.3062
Epoch 1329/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8362 - disc_loss: 0.1718 - rmse: 0.2670 - val_loss: 0.2756
Epoch 1330/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6060 - disc_loss: 0.1523 - rmse: 0.2500 - val_loss: 0.2672
Epoch 1331/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5291 - disc_loss: 0.1614 - rmse: 0.2880 - val_loss: 0.3043
Epoch 1332/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7867 - disc_loss: 0.1601 - rmse: 0.2603 - val_loss: 0.3014
Epoch 1333/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9311 - disc_loss: 0.1620 - rmse: 0.2217 - val_loss: 0.2314
Epoch 1385/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6988 - disc_loss: 0.1625 - rmse: 0.2799 - val_loss: 0.2653
Epoch 1386/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5473 - disc_loss: 0.1677 - rmse: 0.2317 - val_loss: 0.2891
Epoch 1387/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1820 - disc_loss: 0.1598 - rmse: 0.3074 - val_loss: 0.2744
Epoch 1388/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5700 - disc_loss: 0.1702 - rmse: 0.2399 - val_loss: 0.8512
Epoch 1389/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5796 - disc_loss: 0.1661 - rmse: 0.2569 - val_loss: 0.3217
Epoch 1390/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5262 - disc_loss: 0.1633 - rmse: 0.2594 - val_loss: 0.2774
Epoch 1391/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8488 - disc_loss: 0.1570 - rmse: 0.2961 - val_loss: 0.2726
Epoch 1443/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1225 - disc_loss: 0.1648 - rmse: 0.2553 - val_loss: 1.1862
Epoch 1444/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4901 - disc_loss: 0.1615 - rmse: 0.2833 - val_loss: 0.3222
Epoch 1445/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.2564 - disc_loss: 0.1608 - rmse: 0.2620 - val_loss: 0.2882
Epoch 1446/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4469 - disc_loss: 0.1680 - rmse: 0.6275 - val_loss: 0.2688
Epoch 1447/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8465 - disc_loss: 0.1594 - rmse: 0.6841 - val_loss: 0.2941
Epoch 1448/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7174 - disc_loss: 0.1585 - rmse: 0.2330 - val_loss: 0.2793
Epoch 1449/2000
1/1 [========

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [108]:
print(real_df_all.shape)
rnn_target_column = 'tn'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]


(52608, 36)
tn


In [109]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (36825, 36) val_df.shape :  (10522, 36) test_df.shape: (5261, 36)


In [110]:
#fake_df.shape[0]/8760

In [111]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [112]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [113]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [114]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  5
out_num_features :  1


In [115]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [116]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/han/models/nitrogen/


In [117]:
rnn_epochs

20

## 모델 학습

In [118]:
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/20
10/10 [==============================] - ETA: 0s - loss: 0.9717 - mean_absolute_error: 0.6930 - nse: 0.0030 
Epoch 00001: val_loss improved from inf to 0.43942, saving model to save/best_model.h5
10/10 [==============================] - 8s 799ms/step - loss: 0.9717 - mean_absolute_error: 0.6930 - nse: 0.0030 - val_loss: 0.4394 - val_mean_absolute_error: 0.5384 - val_nse: -0.4351
Epoch 2/20
10/10 [==============================] - ETA: 0s - loss: 0.4139 - mean_absolute_error: 0.4368 - nse: 0.5583
Epoch 00002: val_loss improved from 0.43942 to 0.11248, saving model to save/best_model.h5
10/10 [==============================] - 8s 768ms/step - loss: 0.4139 - mean_absolute_error: 0.4368 - nse: 0.5583 - val_loss: 0.1125 - val_mean_absolute_error: 0.2626 - val_nse: 0.6317
Epoch 3/20
10/10 [==============================] - ETA: 0s - loss: 0.3313 - mean_absolute_error: 0.3758 - nse: 0.6703
Epoch 00003: val_loss improved from 0.11248 to 0.09077, saving model to save/best_model.h5
10

## core / window.py / 

In [119]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  1.6701715
predict_day :  5
------------------------
0.07310696 0.00959133 0.097935334
nse3 :  [0.72455335]
pbias3 :  [-2.054332]
save/han/models/nitrogen/
year : 2015 ~ 2020
run :  range(0, 1)
target :  tn
length :  52608
train=7, test=1, val=2
---------------
[0.72455335]
[2.054332]


In [52]:
#train_df.shape, val_df.shape, test_df.shape

In [28]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************            
#                   SSSSSSSS               ******            ******              *****    *****                      
#        SSSS         S  SSS               ******            ******              *****    *****                     
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
